<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/face_attribute_aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python matplotlib tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [11]:
!pip install ultralytics opencv-python matplotlib tensorflow

import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import os
import requests

def download_file(url, filename):
    """Download a file from a URL and save it locally."""
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
        print(f"Downloaded {filename} successfully.")
    else:
        raise Exception(f"Failed to download {filename}. Status code: {response.status_code}")

def preprocess_input(x, v2=True):
    """Preprocess image for model input."""
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

# Download pretrained models
yolo_model_url = "https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt"
gender_model_url = "https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5"
age_model_url = "https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5"

download_file(yolo_model_url, "yolov8n.pt")
download_file(gender_model_url, "gender_model.h5")
download_file(age_model_url, "age_model.h5")

# Verify file sizes (optional but recommended)
print("Verifying file sizes...")
print(f"YOLO model size: {os.path.getsize('yolov8n.pt') / 1024 / 1024:.2f} MB")
print(f"Gender model size: {os.path.getsize('gender_model.h5') / 1024 / 1024:.2f} MB")
print(f"Age model size: {os.path.getsize('age_model.h5') / 1024 / 1024:.2f} MB")

# Load models
try:
    face_detector = YOLO('yolov8n.pt')  # General YOLO model for person detection
    gender_classifier = load_model('gender_model.h5')
    age_predictor = load_model('age_model.h5')
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise

# Configuration
IMAGE_DIR = '/content/images'  # Replace with your image directory
OUTPUT_DIR = '/content/output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

for img_name in os.listdir(IMAGE_DIR):
    img_path = os.path.join(IMAGE_DIR, img_name)
    if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    # Read and convert image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
        continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect people using YOLO
    results = face_detector(img_rgb)

    # Extract bounding boxes
    faces = []
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        for box in boxes:
            if result.names[int(box[5])] == 'person':  # Filter for person class
                x1, y1, x2, y2 = map(int, box[:4])
                faces.append((x1, y1, x2, y2))

    # Initialize counters
    male_count = 0
    female_count = 0
    ages = []

    # Process each detected face
    for (x1, y1, x2, y2) in faces:
        face_region = img_rgb[y1:y2, x1:x2]
        if face_region.size == 0:
            continue

        # Preprocess face
        resized_face = cv2.resize(face_region, (64, 64))
        processed_face = preprocess_input(resized_face)
        face_input = np.expand_dims(processed_face, axis=0)

        # Gender prediction
        gender_probs = gender_classifier.predict(face_input)[0]
        gender = "Male" if np.argmax(gender_probs) == 0 else "Female"
        if gender == "Male":
            male_count += 1
        else:
            female_count += 1

        # Age prediction
        age = int(age_predictor.predict(face_input)[0][0] * 100)
        ages.append(age)

        # Draw annotations
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{gender}, {age}"
        cv2.putText(img, label, (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # Visualization
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Gender Distribution - Male: {male_count}, Female: {female_count}\nAges: {ages}")
    plt.axis('off')
    plt.show()

    # Save processed image
    output_path = os.path.join(OUTPUT_DIR, img_name)
    cv2.imwrite(output_path, img)

# Save models in HDF5 format
gender_classifier.save('gender_model.h5')
age_predictor.save('age_model.h5')

print("Processing complete! Models saved successfully.")

Downloaded yolov8n.pt successfully.
Downloaded gender_model.h5 successfully.
Downloaded age_model.h5 successfully.
Verifying file sizes...
YOLO model size: 6.23 MB
Gender model size: 512.27 MB
Age model size: 513.81 MB
Error loading models: No model config found in the file at gender_model.h5.


ValueError: No model config found in the file at gender_model.h5.